In [ ]:
from mdgen.parsing import parse_train_args

from mdgen.logger import get_logger
logger = get_logger(__name__)

import torch, os
from mdgen.dataset import EquivariantTransformerDataset_MaterialProject



torch.set_float32_matmul_precision('medium')
# args = parse_train_args()
    

trainset = EquivariantTransformerDataset_MaterialProject("data/MP_LixSiy_sims", 6, num_species=94, localmask=False, sim_condition=False, stage="save", save_dir="data/MP_LixSiy_data")

In [ ]:
'''
num_data = len(trainset)
for i in range(num_data):
    len_traj = trainset[i]
    print(i, num_data, len_traj)
    # print(sample_traj.keys())
    # print(sample_traj["x"].shape)
    # print((sample_traj["cell"]).shape)

'''

In [ ]:
print(trainset[0])

In [ ]:
batch_size = 1
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    num_workers=0,
    shuffle=True,
)
sample_batch = next(iter(train_loader))

In [ ]:
from mdgen.parsing import parse_train_args

from mdgen.logger import get_logger
logger = get_logger(__name__)

import torch, os, wandb
from mdgen.dataset import MDGenDataset
from mdgen.wrapper import NewMDGenWrapper
from pytorch_lightning.callbacks import ModelCheckpoint, ModelSummary
import pytorch_lightning as pl


torch.set_float32_matmul_precision('medium')
# args = parse_train_args()

from argparse import Namespace

args = Namespace(ckpt=None, validate=False, num_workers=4, epochs=10000, overfit=False, overfit_peptide=None, overfit_frame=False, train_batches=None, val_batches=None, val_repeat=25, inference_batches=0, batch_size=128, val_freq=None, val_epoch_freq=1, no_validate=False, designability_freq=1, print_freq=100, ckpt_freq=40, wandb=False, run_name='default', accumulate_grad=1, grad_clip=1.0, check_grad=False, grad_checkpointing=False, adamW=False, ema=False, ema_decay=0.999, lr=0.0001, precision='32-true', train_split='splits/4AA_train.csv', val_split='splits/4AA_val.csv', data_dir='data/4AA_data/', num_frames=100, crop=4, suffix="", atlas=False, copy_frames=False, no_pad=False, short_md=False, design_key_frames=False, no_aa_emb=False, no_torsion=False, no_design_torsion=False, supervise_no_torsions=False, supervise_all_torsions=False, no_offsets=False, no_frames=False, hyena=False, no_rope=False, dropout=0.0, scale_factor=1.0, interleave_ipa=False, prepend_ipa=True, oracle=False, num_layers=5, embed_dim=64, mha_heads=16, ipa_heads=4, ipa_head_dim=32, ipa_qk=8, ipa_v=8, time_multiplier=100.0, abs_pos_emb=True, abs_time_emb=False, path_type='GVP', prediction='velocity', sampling_method='dopri5', alpha_max=8, discrete_loss_weight=0.5, dirichlet_flow_temp=1.0, allow_nan_cfactor=False, tps_condition=True, design=False, design_from_traj=False, sim_condition=False, inpainting=False, dynamic_mpnn=False, mpnn=False, frame_interval=None, cond_interval=None)

args.sim_condition=True
args.data_dir="data/CrCoNi_data" 
args.num_frames=1

args.crop=4 
args.ckpt_freq = 40 
args.val_repeat = 25 
args.epochs = 10
args.num_species = 5

args.edge_dim = 4
args.num_convs = 5
args.num_heads = 4
args.ff_dim = 16

args.cutoff= 4
args.localmask = False
args.path_type = "Linear"
args.batch_size = batch_size
# args.prediction='score'
# args.sampling_method = "Euler"

args.design = False
args.potential_model = True

os.environ["MODEL_DIR"] = os.path.join("workdir", args.run_name)

In [ ]:
from mdgen.equivariant_wrapper import EquivariantMDGenWrapper
model = EquivariantMDGenWrapper(args)

In [ ]:
import time

model.iter_step += 1
model.stage = "train"
start1 = time.time()
prep = model.prep_batch(sample_batch)
start = time.time()
print("prep time:", start-start1)

In [ ]:
print(prep.keys())
print(prep["model_kwargs"].keys())

In [ ]:
print(prep["model_kwargs"]["conditions"])

In [ ]:
print(prep["model_kwargs"]["conditions"]["x"].shape)
print(prep["model_kwargs"]["conditions"]["mask"].shape)
raise RuntimeError

## Test graph algo

In [ ]:
from mdgen.model.utils.data_utils import (
    frac_to_cart_coords_with_lattice,
    get_pbc_distances,
    lattice_params_to_matrix_torch,
    radius_graph_pbc,
)
from mdgen.model.gemnet.utils import (
    repeat_blocks,
)
otf_graph = True

In [ ]:

# edge_index, to_jimages, num_bonds = radius_graph_pbc(
#     cart_coords=prep['latents'].view(-1, 3),
#     lattice=prep['model_kwargs']['cell'].view(-1, 3, 3),
#     num_atoms=prep['model_kwargs']['num_atoms'].view(-1),
#     radius=model.model.cutoff,
#     max_num_neighbors_threshold=50,
#     max_cell_images_per_dim=5,
# )


In [ ]:
t, x0, x1 = model.transport.sample(prep['latents'])
t, xt, ut = model.transport.path_sampler.plan(t, x0, x1)

In [ ]:
# print(prep['latents'][0].unsqueeze(0).shape)
# v_latents = prep['latents'][0].unsqueeze(0).expand(batch_size,-1,-1,-1)
# print(v_latents.shape)
# print(prep['model_kwargs']['cell'][0].unsqueeze(0).shape)
# v_cell = prep['model_kwargs']['cell'][0].unsqueeze(0).expand(batch_size,-1,-1,-1)
# print(v_cell.shape)
# print(prep['model_kwargs']['num_atoms'][0].unsqueeze(0).shape)
# v_num_atoms = prep['model_kwargs']['num_atoms'][0].unsqueeze(0).expand(batch_size,-1)
# print(v_num_atoms.shape)
# print(prep['species'][0].unsqueeze(0).shape)
# v_species = prep['species'][0].unsqueeze(0).expand(batch_size,-1,-1,-1)
# print(v_species.shape)

In [ ]:

# edge_index, to_jimages, num_bonds = radius_graph_pbc(
#     cart_coords=v_latents.reshape(-1, 3),
#     lattice=v_cell.view(-1, 3, 3),
#     num_atoms=v_num_atoms.view(-1),
#     radius=model.model.cutoff,
#     max_num_neighbors_threshold=50,
#     max_cell_images_per_dim=5,
# )

In [ ]:

conditions = prep["model_kwargs"]["conditions"]
edge_index_cond, to_jimages_cond, num_bonds_cond = radius_graph_pbc(
    cart_coords=conditions["x"].view(-1, 3),
    lattice=conditions["cell"].view(-1, 3, 3),
    num_atoms=conditions["num_atoms"].view(-1),
    radius=model.model.cutoff,
    max_num_neighbors_threshold=model.model.max_num_neighbors_threshold,
    max_cell_images_per_dim=model.model.max_cell_images_per_dim,
)


In [ ]:
# out = get_pbc_distances(
#     prep['latents'].view(-1, 3),
#     edge_index,
#     prep['model_kwargs']['cell'].view(-1, 3, 3),
#     to_jimages,
#     prep['model_kwargs']['num_atoms'].view(-1),
#     num_bonds,
#     coord_is_cart=True,
#     return_offsets=True,
#     return_distance_vec=True,
# )

In [ ]:
# out = get_pbc_distances(
#     v_latents.reshape(-1, 3),
#     edge_index,
#     v_cell.view(-1, 3, 3),
#     to_jimages,
#     v_num_atoms.view(-1),
#     num_bonds,
#     coord_is_cart=True,
#     return_offsets=True,
#     return_distance_vec=True,
# )

In [ ]:
out_cond = get_pbc_distances(
    conditions["x"].view(-1, 3),
    edge_index_cond,
    conditions["cell"].view(-1, 3, 3),
    to_jimages_cond,
    conditions["num_atoms"].view(-1),
    num_bonds_cond,
    coord_is_cart=True,
    return_offsets=True,
    return_distance_vec=True,
)
out_cond["species"] = conditions["species"]
out_cond["mask"] = conditions["mask"]

In [ ]:
print(t)

In [ ]:
# edge_index = out["edge_index"]
# edge_len = out["distances"]
# edge_vec = out["distance_vec"]
# edge_attr = torch.hstack([edge_vec, edge_len.view(-1, 1)])
# B,T,L,_ = v_species.shape
# t = t.unsqueeze(-1).unsqueeze(1).expand(-1,T,-1).unsqueeze(2).expand(-1,-1,L,-1)
# h,v,_edge_attr = model.model.encoder(v_species.reshape(-1,5), edge_index, edge_attr, edge_vec, t.reshape(-1,1), )

In [ ]:
if out_cond is not None:
    edge_index_cond = out_cond["edge_index"]
    edge_len_cond = out_cond["distances"]
    edge_vec_cond = out_cond["distance_vec"]
    edge_attr_cond = torch.hstack([edge_vec_cond, edge_len_cond.view(-1, 1)])
    species_cond = out_cond["species"]

    h_cond, v_cond, edge_attr_cond = model.model.encoder(
        species_cond.view(-1,5), 
        edge_index_cond, edge_attr_cond, edge_vec_cond, 
        torch.zeros([*species_cond.shape[:-1],1], device=species_cond.device).reshape(-1,1))
    print(h_cond.shape)
    print(torch.where(out_cond["mask"].reshape(-1,1)))
    # for i in range()
    for i in range(h_cond.shape[0]):
        print(i, (model.model.cond_to_emb(h_cond)*(out_cond["mask"].reshape(-1,1)))[i])

In [ ]:
edge_index = out["edge_index"]
edge_len = out["distances"]
edge_vector = out["distance_vec"]


In [ ]:
print(edge_index.shape)

In [ ]:
'''
# !!! need to keep track of index exchanges and take into consideration when inferencing to use this
edge_index, cell_offsets, neighbors, edge_len, edge_vec = reorder_symmetric_edges(
        edge_index,
        to_jimages,
        num_bonds,
        out["distances"],
        out["distance_vec"],
    )
'''

In [ ]:
print(out.keys())

## test equivariant_latent_model for energy and forces

In [ ]:
print(prep.keys())
t = torch.ones(batch_size)

In [ ]:
energy = model.model(prep["latents"], t, **prep["model_kwargs"] )

In [ ]:
print(energy)
print(energy.sum(dim=2))

In [ ]:
loss_energy = (energy - prep["E"])**2
print(loss_energy)

## test equivariant_latent_model

In [ ]:
t, x0, x1 = model.transport.sample(prep['latents'])
# t, xt, ut = model.transport.path_sampler.plan(t, x0, x1)


In [ ]:
print(model.transport.path_sampler)

In [ ]:
print(prep["model_kwargs"]["conditions"]['mask'].reshape(-1).shape)

In [ ]:
print(model.model.mask_to_emb(prep["model_kwargs"]["conditions"]['mask'].reshape(-1)).shape)

In [ ]:
model_output = model.model(prep['latents'], t, **prep['model_kwargs'])

In [ ]:
out_dict = model.transport.training_losses(
    model=model.model,
    x1=prep['latents'],
    aatype1=prep['species'],
    mask=prep['loss_mask'],
    model_kwargs=prep['model_kwargs']
)
# loss = out_dict["loss"].mean()
print(out_dict)

In [ ]:
print(prep['model_kwargs']["conditions"])

## Training

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.model.parameters(), lr=0.001)
loss_epochs = []

In [ ]:
print(model.transport.model_type)

In [ ]:
# Training loop

num_epochs = 20
for epoch in range(num_epochs):
    Step = 0
    # for batch in train_loader:
    #     # Forward pass
    #     batch_ = {key: value.to(device) if isinstance(value, torch.Tensor) else value 
    #             for key, value in batch.items()}
    #     prep = model.prep_batch(batch_)
    # out_dict = model.transport.training_losses(
    #     model=model.model,
    #     x1=prep['latents'],
    #     aatype1=prep['species'],
    #     mask=prep['loss_mask'],
    #     model_kwargs=prep['model_kwargs']
    # )
    # loss = out_dict["loss"].mean()
    t = torch.ones(batch_size)
    energy = model.model(prep["latents"], t, **prep["model_kwargs"] )
    loss = ((energy - prep["E"])**2).mean()
    
    # Backward pass and optimize
    optimizer.zero_grad()
    # print("Starting backward")
    loss.backward()
    # print("Done")
    optimizer.step()
    Step += 1
    loss_epochs.append(loss.item())
    # print(f'Epoch [{epoch+1}/{num_epochs}], Step [{Step}], Loss: {loss.item():.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(4,3.5))
plt.plot(loss_epochs)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.semilogy()

## Testing the trained model

In [ ]:
t, x0, x1 = model.transport.sample(prep['latents'])
t, xt, ut = model.transport.path_sampler.plan(t, x0, x1)
out_dict = model.transport.training_losses(
    model=model.model,
    x1=prep['latents'],
    aatype1=prep['species'],
    mask=prep['loss_mask'],
    model_kwargs=prep['model_kwargs']
)
loss_last = out_dict["loss_continuous"][torch.where(prep['loss_mask'].squeeze(-1)!=0)].detach().mean(dim=1)
loss_fkl_last = out_dict["loss_fisherreg"]
print(loss_last)
import matplotlib.pyplot as plt
plt.figure(figsize=(4,3.5))
plt.scatter(t.detach().cpu().numpy(), loss_last.detach().cpu().numpy())
plt.scatter(t.detach().cpu().numpy(), loss_fkl_last.detach().cpu().numpy())
plt.xlabel("Diffusion time")
plt.ylabel("Loss")

In [ ]:
plt.figure(figsize=(4,3.5))
plt.scatter(t.detach().cpu().numpy(), loss_last.detach().cpu().numpy())
plt.xlabel("Diffusion time")
plt.ylabel("Loss")